# **VSC 에서 OpenAI Whisper 사용하기**



---



- 💡 **NOTE**
    - 이 과정은 PC Visual Studio Code(이하 VSC)에서 진행합니다.
    - 가상환경 생성과 라이브러리 설치는 VSC 터미널에서 진행합니다.
    - 아래 내용은 Windows PC 환경을 가정하여 설명되었습니다.    
    - 참고 : https://bcuts.tistory.com/192





---



## **Whisper 소개**
- Whisper는 OpenAI가 개발한 자동 음성 인식(ASR: Automatic Speech Recognition) 시스템
- 2022년 9월 OpenAI에 의해 처음 공개한 오픈 소스 모델
- 인터넷에서 수집한 방대한 양의 오디오 데이터로 훈련되어, 다양한 언어와 악센트, 배경 소음 속에서도 매우 뛰어난 정확도를 자랑함
- 기존의 음성 인식 모델들이 가진 한계를 극복하고, 음성-텍스트 변환과 언어 번역을 동시에 수행할 수 있도록 설계된 혁신적인 모델
- 대규모 약한 지도 학습(Large-Scale Weak Supervision)"이라는 독특한 방식을 사용
    - Whisper는 인터넷에서 수집한 방대한 양의 오디오와 해당 오디오의 전사본으로 학습
    - 이 데이터들은 유튜브 영상의 자동 생성 자막처럼 정확도가 완벽하지는 않지만, 양이 엄청나게 많다는 특징을 가짐
    - 대량의 "약한" 데이터로 학습함으로써, Whisper는 다양한 언어, 배경 소음, 억양 등 현실 세계의 복잡한 음성 환경에 대한 높은 일반화 능력을 갖추게 됨

- transcribe : 음성 인식
    > model.transcribe(audio, fp16=False)
- translate : 번역  
    > model.translate(audio, fp16=False)

----

## **설치 순서(Windows)**

1. 가상환경 생성 (권장)
2. PyTorch 설치
3. Whisper 설치

### **1.가상환경 생성 (권장)**
터미널에서 실행한다.


In [ ]:
python -m venv whisper-env
whisper-env\Scripts\activate

- **만약 보안 오류가 발생한 경우(UnauthorizedAccess)**
- 해결 방법(영구해결)

1) VS Code 터미널(파워셸)에서 실행

In [ ]:
# 터미널에서 실행한다.
Set-ExecutionPolicy -Scope CurrentUser -ExecutionPolicy RemoteSigned

2) 새 터미널을 열고 가상환경 활성화

In [ ]:
# 터미널에서 실행한다.
.\whisper-env\Scripts\Activate.ps1

3) 확인

In [ ]:
# 터미널에서 실행한다.
python -c "import sys; print(sys.executable); print(sys.version)"

### **2.PyTorch 설치**

In [14]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


### **3.Whisper 설치**

In [15]:
pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to c:\users\user\appdata\local\temp\pip-req-build-lq51dhdd
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-lq51dhdd'


### **4.ffmpeg 설치**
- 참고 : https://angelplayer.tistory.com/351
- https://ffmpeg.org/  
- 다운로드 > 윈도우 버튼 >(Windows EXE Files) Windows builds from gyan.dev 선택
- ffmpeg-git-full.7z 선택 > 압축풀고 > 폴더 이름을 ffmpeg 으로 변경한다.
- ffmpeg 폴더 전체를 C:\로 이동시키고
- 고급 시스템 설정 > 환경변수 > 시스템 변수 > Path에 추가한다. (C:\ffmpeg\bin)
- 꼭!!! VSC 재시작한다.

In [16]:
# 사전 점검 코드
from pathlib import Path
import shutil, sys

audio = Path(r"./audio/오디오-슬라이드2.mp3")  # 필요시 절대경로로 바꾸세요
print("▶ CWD:", Path.cwd())
print("▶ Audio Path:", audio.resolve())
print("▶ Audio Exists?:", audio.exists())
print("▶ ffmpeg on PATH?:", shutil.which("ffmpeg"))
print("▶ ffprobe on PATH?:", shutil.which("ffprobe"))

▶ CWD: c:\python\ktcloud\ktcloud_genai
▶ Audio Path: C:\python\ktcloud\ktcloud_genai\audio\오디오-슬라이드2.mp3
▶ Audio Exists?: True
▶ ffmpeg on PATH?: C:\ffmpeg\bin\ffmpeg.EXE
▶ ffprobe on PATH?: C:\ffmpeg\bin\ffprobe.EXE


In [ ]:
# - 터미널에서 실행
$env:Path = "C:\ffmpeg\bin;" + $env:Path
ffmpeg -version

In [17]:
from pathlib import Path
print("CWD:", Path.cwd())
print("Rel exists?:", Path("./audio/오디오-슬라이드2.mp3").exists())
print("Abs exists?:", Path(r"C:\python\ktcloud\ktcloud_genai\audio\오디오-슬라이드2.mp3").exists())

CWD: c:\python\ktcloud\ktcloud_genai
Rel exists?: True
Abs exists?: True




---



## **Whisper로 음성 파일을 텍스트로 변환하기**

1. Whisper 라이브러리 불러오기
2. 모델 로드하기
3. 음성 파일을 읽어들여 변환
4. 결과 출력 또는 저장

- 영어 오디오

In [18]:
# 1.Whisper 라이브러리 불러오기
import whisper

# 2.모델 로드하기
model = whisper.load_model("small")  # tiny, base, small, medium, large

# 3.음성 파일을 읽어들여 변환 .wav, .mp3, .m4a, .webm, .ogg, .flac 등
audio = r"C:\python\ktcloud\ktcloud_genai\audio\아동-영어1.wav"
result = model.transcribe(audio, fp16=False)  # fp16=False 옵션은 CPU 환경에서 필요
print(result["text"])


# 4.결과 출력 또는 저장
with open("output_eng.txt", "w", encoding="utf-8") as f:
    f.write(result["text"])

 Pay attention to me.


- 한글 오디오

In [19]:
import whisper, shutil
from pathlib import Path


audio_path = Path(r"C:\python\ktcloud\ktcloud_genai\audio\오디오-슬라이드2.mp3")  # 절대경로 권장


# 1) 필수 전제 확인
assert audio_path.exists(), f"오디오 파일이 없습니다: {audio_path}"
assert shutil.which("ffmpeg"), "ffmpeg를 찾을 수 없습니다. 설치 후 PATH에 등록하세요."
assert shutil.which("ffprobe"), "ffprobe를 찾을 수 없습니다. ffmpeg 설치 시 함께 등록됩니다."


# 2) 모델 로드 (CPU 사용 시 fp16 비활성화 권장)
model = whisper.load_model("small")   # tiny/base/small/medium/large
result = model.transcribe(str(audio_path), fp16=False)  # CPU면 fp16=False로 경고 억제
print(result["text"])


# 3) 저장
Path("output_kor.txt").write_text(result["text"], encoding="utf-8")
print("저장 완료: output_kor.txt")


 커피의 맛을 이해하기 위해서는 4가지 주요 요소를 살펴보는 것이 중요합니다. 첫째, 삼미는 심맛의 강도를 나타내며 이는 과일이나 로스팅 과정에서 기인합니다. 둘째, 바디감은 커피가 입 안에서 느껴지는 묵직함과 질감을 의미하며 커피의 무게감을 표현합니다.
저장 완료: output_kor.txt


----

## **실시간 음성 인식**

- 참고 : https://bcuts.tistory.com/197

- 반복 과정
    - 마이크로부터 5초간 음성
    - 입력 데이터를 .wav 파일로 저장
    - 저장된 파일을 Whisper로 처리
    - 텍스트 출력

### 라이브러리 설치

In [20]:
!pip install pyaudio numpy soundfile

### 실시간 음성인식 코드

In [21]:
import whisper
import pyaudio
import wave
import time

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "temp.wav"

model = whisper.load_model("base")
audio = pyaudio.PyAudio()

stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

print("실시간 음성 인식 시작 (Ctrl+C로 종료)")

try:
    while True:
        frames = []
        for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)

        wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()

        result = model.transcribe(WAVE_OUTPUT_FILENAME)
        print("📢 인식된 텍스트:", result["text"])

except KeyboardInterrupt:
    print("종료합니다.")
    stream.stop_stream()
    stream.close()
    audio.terminate()



실시간 음성 인식 시작 (Ctrl+C로 종료)
📢 인식된 텍스트: 
📢 인식된 텍스트:  점순의 먹은 라면은 맛있었다
종료합니다.


----